This notebook will contain code relevant to the logistic regression model.  

# Load files and Import packages

In [1]:
## Math
from math import floor
from statistics import *

## Handling Arrays and Dataframes
import pandas as pd
import numpy as np

## Train Test Set Splitting
from sklearn.model_selection import train_test_split

## Sampling
from imblearn.over_sampling import SMOTENC
from sklearn.utils import resample

## Feature Scaling
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

## Model Train and Testing
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as met
import matplotlib.pyplot as plt

## PCA
from sklearn.decomposition import PCA

## Fine-tuning
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

## LIME
!pip install lime
import lime
import lime.lime_tabular

In [2]:
## mount drive and retrieve labelled data

from google.colab import drive
drive.mount("/content/drive")

import os
os.chdir("/content/drive/MyDrive/DSA4266_Tundra")

m_data_raw = pd.read_csv("merged_data.csv")
#m_data_raw = m_data_raw.iloc[:,1:]
m_data_raw.reset_index(drop = True, inplace = True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
m_data_raw.head()

,transcript_id,transcript_position,sequence,-1_flank_length,-1_flank_std,-1_flank_mean,central_length,central_std,central_mean,+1_flank_length,+1_flank_std,+1_flank_mean,gene_id,label
0,ENST00000000233,244,AAGACCA,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1,ENSG00000004059,0
1,ENST00000000233,244,AAGACCA,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9,ENSG00000004059,0
2,ENST00000000233,244,AAGACCA,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6,ENSG00000004059,0
3,ENST00000000233,244,AAGACCA,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4,ENSG00000004059,0
4,ENST00000000233,244,AAGACCA,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2,ENSG00000004059,0


In [4]:
m_data_raw['transcript_id'].nunique()

5333

In [5]:
m_data_raw['gene_id'].nunique()

3852

In [6]:
m_data_raw['sequence'].nunique()

288

# Data Preprocessing

## Remove correlated variables

In [7]:
m_data_raw.head()

,transcript_id,transcript_position,sequence,-1_flank_length,-1_flank_std,-1_flank_mean,central_length,central_std,central_mean,+1_flank_length,+1_flank_std,+1_flank_mean,gene_id,label
0,ENST00000000233,244,AAGACCA,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1,ENSG00000004059,0
1,ENST00000000233,244,AAGACCA,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9,ENSG00000004059,0
2,ENST00000000233,244,AAGACCA,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6,ENSG00000004059,0
3,ENST00000000233,244,AAGACCA,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4,ENSG00000004059,0
4,ENST00000000233,244,AAGACCA,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2,ENSG00000004059,0


In [8]:
corr = m_data_raw.corr()
corr.style.background_gradient(cmap='coolwarm')

<ipython-input-8-9d2b1d4d835e>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = m_data_raw.corr()


,transcript_position,-1_flank_length,-1_flank_std,-1_flank_mean,central_length,central_std,central_mean,+1_flank_length,+1_flank_std,+1_flank_mean,label
transcript_position,1.000000,-0.010329,-0.047637,-0.079866,-0.020504,-0.045754,-0.068146,-0.006597,-0.029134,0.025372,0.018985
-1_flank_length,-0.010329,1.000000,0.127075,0.094277,0.001115,0.009783,-0.029552,0.014739,-0.018274,-0.012424,0.009458
-1_flank_std,-0.047637,0.127075,1.000000,0.380163,0.005668,0.059138,-0.030368,0.012969,0.015944,-0.024378,0.024992
-1_flank_mean,-0.079866,0.094277,0.380163,1.000000,0.069187,0.057715,0.239609,0.016304,0.121022,0.008591,0.099535
central_length,-0.020504,0.001115,0.005668,0.069187,1.000000,0.128894,0.150908,0.035523,0.055304,-0.055893,0.006456
central_std,-0.045754,0.009783,0.059138,0.057715,0.128894,1.000000,0.438944,0.028204,0.147590,-0.429502,-0.000377
central_mean,-0.068146,-0.029552,-0.030368,0.239609,0.150908,0.438944,1.000000,0.051361,0.392869,-0.247272,0.073311
+1_flank_length,-0.006597,0.014739,0.012969,0.016304,0.035523,0.028204,0.051361,1.000000,0.095774,-0.003797,0.011718
+1_flank_std,-0.029134,-0.018274,0.015944,0.121022,0.055304,0.147590,0.392869,0.095774,1.000000,-0.078569,0.007753
+1_flank_mean,0.025372,-0.012424,-0.024378,0.008591,-0.055893,-0.429502,-0.247272,-0.003797,-0.078569,1.000000,0.039638


## Label Encode categorical data

In [9]:
print(m_data_raw.shape)

(11027106, 14)


In [10]:
pip install category_encoders

In [11]:
import category_encoders as ce

encoder = ce.CountEncoder()

In [12]:
# m_data_ce = m_data_raw.copy()

# cateogorical_list = ["transcript_id", "gene_id", "sequence"]

# for col in cateogorical_list:

#     holder = m_data_ce[col].values.reshape(-1, 1)

#     temp = pd.DataFrame(encoder.fit_transform(holder))

#     temp.columns = encoder.get_feature_names_out([col])

#     m_data_ce = m_data_ce.join(temp)

In [13]:
m_data_raw.head()

,transcript_id,transcript_position,sequence,-1_flank_length,-1_flank_std,-1_flank_mean,central_length,central_std,central_mean,+1_flank_length,+1_flank_std,+1_flank_mean,gene_id,label
0,ENST00000000233,244,AAGACCA,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1,ENSG00000004059,0
1,ENST00000000233,244,AAGACCA,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9,ENSG00000004059,0
2,ENST00000000233,244,AAGACCA,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6,ENSG00000004059,0
3,ENST00000000233,244,AAGACCA,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4,ENSG00000004059,0
4,ENST00000000233,244,AAGACCA,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2,ENSG00000004059,0


## Split data into Train-Test (Stratified 80/20)

In [14]:
y = m_data_raw[["label"]] # labels

X = m_data_raw.copy() # features
X.drop("label", axis = 1, inplace = True)
X.drop("sequence", axis = 1, inplace = True)
X.drop("transcript_id", axis = 1, inplace = True)
X.drop("gene_id", axis = 1, inplace = True)

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.20, random_state = 4266, stratify = y)

train_data = train_y.join(train_X)
test_data = test_y.join(test_X)

train_X_og = train_X.copy()
test_X_og = test_X.copy()

In [15]:
print(train_X.shape)
print(test_X.shape)

(8821684, 10)
(2205422, 10)


In [16]:
X

,transcript_position,-1_flank_length,-1_flank_std,-1_flank_mean,central_length,central_std,central_mean,+1_flank_length,+1_flank_std,+1_flank_mean
0,244,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1
1,244,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9
2,244,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6
3,244,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4
4,244,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2
...,...,...,...,...,...,...,...,...,...,...
11027101,1693,0.00418,7.49,108.0,0.00564,10.20,116.0,0.01000,2.01,76.4
11027102,1693,0.00664,1.91,109.0,0.00598,12.30,110.0,0.01760,2.61,74.6
11027103,1693,0.00721,4.58,105.0,0.00398,6.58,113.0,0.00316,2.28,85.3
11027104,1693,0.00266,2.33,109.0,0.00913,10.40,108.0,0.00664,4.44,76.8


## Scale data

In [17]:
## data normalisation - new_x = (x – min) / (max – min)
mm = MinMaxScaler()
train_X = mm.fit_transform(train_X)
test_X = mm.fit_transform(test_X)

# Logistic Regression Model

## Setup of helper function and dataframes to store results


a cross validation helper function below to remember the indexes that make up the best training set during cross validation.

In [18]:
def get_best(training_x, training_y, model):
  skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=4266)
  lst_f1_stratified = []
  best_f1 = 0
  best_x_fold = []
  best_y_fold = []
  for train_index, val_index in skf.split(training_x, training_y):

      x_train_fold, x_val_fold = training_x[train_index], training_x[val_index]
      y_train_fold, y_val_fold = training_y.values.ravel()[train_index], training_y.values.ravel()[val_index]

      if len(best_x_fold) == 0:
        best_x_fold.extend(x_train_fold)
        best_y_fold.extend(y_train_fold)

      lr = model.fit(x_train_fold, y_train_fold)

      current_model_f1 = met.f1_score(y_true = y_val_fold, y_pred = model.predict(x_val_fold))
      lst_f1_stratified.append(current_model_f1)

      if current_model_f1 > best_f1:
        best_x_fold.clear()
        best_y_fold.clear()
        best_x_fold.extend(x_train_fold)
        best_y_fold.extend(y_train_fold)

  print('List of possible F1-score:', [round(x, 5) for x in lst_f1_stratified])
  print('\nMaximum F1-score That can be obtained from this model is:', round(max(lst_f1_stratified)*100, 5), '%')
  print('\nMinimum F1-score:', round(min(lst_f1_stratified)*100, 5), '%')
  print('\nAverage F1-score:', round(mean(lst_f1_stratified), 5))
  print('\nStandard Deviation is:', round(stdev(lst_f1_stratified), 5))

  return (best_x_fold, best_y_fold)

We define two variables to store model performance during training and testing.

In [19]:
## make predictions and compute metrics
# 1) we want to maximise f1 score - 2/(1/P + 1/R)
# 2) cfm.ravel() - tn, fp, fn, tp
# 3) class-specific accuracy for reference

column_template = {"model": [], "roc auc score": [],
                   "F1-score": [], "precision": [], "recall" : [],
                   "tn": [], "fp": [], "fn": [], "tp": [],
                   "class 0 accuracy": [], "class 1 accuracy ": []}

training_results = pd.DataFrame(column_template)
testing_results = pd.DataFrame(column_template)

Model is overly biased towards majority class, does not predict minority class at all. Model does not learn well, leading to poor testing performance subsequently.

## 2) Weighted Model

Make loss function weighted such that misclassifications of minority class examples have higher cost.

Use cross validation to get subset of training data with best F1-score on validation set.

In [20]:
# fit model
clf = LogisticRegression(random_state = 4266, max_iter = 500, class_weight = "balanced")

# CV
best_fold = get_best(train_X, train_y, clf)

List of possible F1-score: [0.13598, 0.13541, 0.13655, 0.13572, 0.13466, 0.13471, 0.13543, 0.13561, 0.13526, 0.13562]

Maximum F1-score That can be obtained from this model is: 13.65494 %

Minimum F1-score: 13.46557 %

Average F1-score: 0.13549

Standard Deviation is: 0.00056


Train model using subset of data identified above using cross validation and Test model using holdout set.

In [21]:
X_train = best_fold[0]
y_train = best_fold[1]
X_test = test_X
y_test = test_y

# fit model
clf = LogisticRegression(random_state = 4266,
                         max_iter = 500, class_weight = "balanced").fit(X_train, y_train)

cfm = met.confusion_matrix(y_true = y_train, y_pred = clf.predict(X_train))
cfm2 = cfm.astype("float") / cfm.sum(axis=1)[:, np.newaxis]
train_1 = {"model": "weighted",
  "roc auc score": met.roc_auc_score(y_true = y_train, y_score = clf.predict(X_train)),
	"F1-score": met.f1_score(y_true = y_train, y_pred = clf.predict(X_train)),
  "precision" : met.precision_score(y_true = y_train, y_pred = clf.predict(X_train)),
  "recall": met.recall_score(y_true = y_train, y_pred = clf.predict(X_train)),
	"tn": cfm.ravel()[0],
	"fp": cfm.ravel()[1],
  "fn": cfm.ravel()[2],
  "tp": cfm.ravel()[3],
  "class 0 accuracy": cfm2.diagonal()[0],
  "class 1 accuracy ": cfm2.diagonal()[1]}
training_results = training_results.append(train_1, ignore_index = True)

cfm = met.confusion_matrix(y_true = y_test, y_pred = clf.predict(X_test))
cfm2 = cfm.astype("float") / cfm.sum(axis=1)[:, np.newaxis]
test_1 = {"model": "weighted",
  "roc auc score": met.roc_auc_score(y_true = y_test, y_score = clf.predict(X_test)),
	"F1-score": met.f1_score(y_true = y_test, y_pred = clf.predict(X_test)),
  "precision": met.precision_score(y_true = y_test, y_pred = clf.predict(X_test)),
  "recall": met.recall_score(y_true = y_test, y_pred = clf.predict(X_test)),
	"tn": cfm.ravel()[0],
	"fp": cfm.ravel()[1],
  "fn": cfm.ravel()[2],
  "tp": cfm.ravel()[3],
  "class 0 accuracy": cfm2.diagonal()[0],
  "class 1 accuracy ": cfm2.diagonal()[1]}
testing_results = testing_results.append(test_1, ignore_index = True)

<ipython-input-21-ffed8b293cae>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  training_results = training_results.append(train_1, ignore_index = True)
<ipython-input-21-ffed8b293cae>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  testing_results = testing_results.append(test_1, ignore_index = True)


Model makes some minority class predictions, correctly identifying 27 bad customers out of 53. Huge improvement from plain model.

In [22]:
training_results

,model,roc auc score,F1-score,precision,recall,tn,fp,fn,tp,class 0 accuracy,class 1 accuracy
0,weighted,0.641428,0.135501,0.075268,0.678334,4582173.0,2997650.0,115701.0,243992.0,0.604522,0.678334


In [23]:
testing_results

,model,roc auc score,F1-score,precision,recall,tn,fp,fn,tp,class 0 accuracy,class 1 accuracy
0,weighted,0.64225,0.134999,0.07484,0.688235,1255440.0,850067.0,31150.0,68765.0,0.596265,0.688235


## 3) Model with undersampled majority class and oversampled minority class

Mitigate class imbalance's effect on model training by resampling more minority class examples and less majority class examples.

In [24]:
train_data.label.value_counts()

0    8422025
1     399659
Name: label, dtype: int64

Sample majority class by a factor of 1/5, sample minority class by a factor of 5.

In [25]:
# Separate majority and minority classes
train_data_major = train_data[train_data.label == 0]
train_data_minor = train_data[train_data.label == 1]

# Downsample majority class
train_data_major_resampled = resample(train_data_major,
                                 replace = False,
                                 n_samples = floor(8422025/5),
                                 random_state = 4266)

# Upsample minority class
train_data_minor_resampled = resample(train_data_minor,
                                 replace = True,
                                 n_samples = 399659*5,
                                 random_state = 4266)

# Combine minority class with downsampled majority class
train_data_resampled = pd.concat([train_data_major_resampled, train_data_minor_resampled])

In [26]:
train_data_resampled.label.value_counts()

1    1998295
0    1684405
Name: label, dtype: int64

In [27]:
y_train_resampled = train_data_resampled["label"]
X_train_resampled = train_data_resampled.drop("label", axis = 1)
X_train_resampled = mm.fit_transform(X_train_resampled)

Use cross validation to get subset of training data with best F1-score on validation set.

In [28]:
# fit model
clf = LogisticRegression(random_state = 3244, max_iter = 500, class_weight = "balanced")

# CV
best_fold = get_best(X_train_resampled, y_train_resampled, clf)

List of possible F1-score: [0.67562, 0.67501, 0.67547, 0.67313, 0.67577, 0.67476, 0.67447, 0.67408, 0.67498, 0.67394]

Maximum F1-score That can be obtained from this model is: 67.5766 %

Minimum F1-score: 67.31336 %

Average F1-score: 0.67472

Standard Deviation is: 0.00083


Train model using subset of data identified above using cross validation and Test model using holdout set.

In [29]:
X_train = best_fold[0]
y_train = best_fold[1]
X_test = test_X
y_test = test_y

# fit model
clf = LogisticRegression(random_state = 3244,
                         max_iter = 500, class_weight = "balanced").fit(X_train, y_train)

cfm = met.confusion_matrix(y_true = y_train, y_pred = clf.predict(X_train))
cfm2 = cfm.astype("float") / cfm.sum(axis=1)[:, np.newaxis]
train_1 = {"model": "weighted and resampled",
  "roc auc score": met.roc_auc_score(y_true = y_train, y_score = clf.predict(X_train)),
	"F1-score": met.f1_score(y_true = y_train, y_pred = clf.predict(X_train)),
  "precision" : met.precision_score(y_true = y_train, y_pred = clf.predict(X_train)),
  "recall": met.recall_score(y_true = y_train, y_pred = clf.predict(X_train)),
	"tn": cfm.ravel()[0],
	"fp": cfm.ravel()[1],
  "fn": cfm.ravel()[2],
  "tp": cfm.ravel()[3],
  "class 0 accuracy": cfm2.diagonal()[0],
  "class 1 accuracy ": cfm2.diagonal()[1]}
training_results = training_results.append(train_1, ignore_index = True)

cfm = met.confusion_matrix(y_true = y_test, y_pred = clf.predict(X_test))
cfm2 = cfm.astype("float") / cfm.sum(axis=1)[:, np.newaxis]
test_1 = {"model": "weighted and resampled",
  "roc auc score": met.roc_auc_score(y_true = y_test, y_score = clf.predict(X_test)),
	"F1-score": met.f1_score(y_true = y_test, y_pred = clf.predict(X_test)),
  "precision": met.precision_score(y_true = y_test, y_pred = clf.predict(X_test)),
  "recall": met.recall_score(y_true = y_test, y_pred = clf.predict(X_test)),
	"tn": cfm.ravel()[0],
	"fp": cfm.ravel()[1],
  "fn": cfm.ravel()[2],
  "tp": cfm.ravel()[3],
  "class 0 accuracy": cfm2.diagonal()[0],
  "class 1 accuracy ": cfm2.diagonal()[1]}
testing_results = testing_results.append(test_1, ignore_index = True)

<ipython-input-29-caddbe332f63>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  training_results = training_results.append(train_1, ignore_index = True)
<ipython-input-29-caddbe332f63>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  testing_results = testing_results.append(test_1, ignore_index = True)


During training, model precision and recall improved, suggesting that model learnt more patterns to distinguish the two classes. However, testing results show that this may not be the case; we may have overfitted the model to the resampled training data which has lost significant signal information, leading to worse testing performance.

In [30]:
training_results

,model,roc auc score,F1-score,precision,recall,tn,fp,fn,tp,class 0 accuracy,class 1 accuracy
0,weighted,0.641428,0.135501,0.075268,0.678334,4582173.0,2997650.0,115701.0,243992.0,0.604522,0.678334
1,weighted and resampled,0.641909,0.674865,0.670909,0.678869,917083.0,598882.0,577543.0,1220922.0,0.604950,0.678869


In [32]:
testing_results

,model,roc auc score,F1-score,precision,recall,tn,fp,fn,tp,class 0 accuracy,class 1 accuracy
0,weighted,0.642250,0.134999,0.07484,0.688235,1255440.0,850067.0,31150.0,68765.0,0.596265,0.688235
1,weighted and resampled,0.641816,0.136121,0.07570,0.674373,1282799.0,822708.0,32535.0,67380.0,0.609259,0.674373


## 4) Model with undersampled majority class and SMOTE-oversampled minority class

Attempt less aggressive majority class downsampling and use SMOTE to increase minority class examples in training data.

In [33]:
train_data.label.value_counts()

0    8422025
1     399659
Name: label, dtype: int64

Sample majority class by a factor of 2/5

In [34]:
# Separate majority and minority classes
train_data_major = train_data[train_data.label == 0]
train_data_minor = train_data[train_data.label == 1]

# Downsample majority class
train_data_major_resampled = resample(train_data_major,
                                 replace = False,
                                 n_samples = floor(8422025*2/5),
                                 random_state = 4266)
# Combine
train_data_resampled = pd.concat([train_data_major_resampled, train_data_minor])

In [35]:
train_data_resampled.label.value_counts()

0    3368810
1     399659
Name: label, dtype: int64

In [36]:
y_train_resampled = train_data_resampled["label"]
X_train_resampled = train_data_resampled.drop("label", axis = 1)
X_train_resampled = mm.fit_transform(X_train_resampled)

Identify categorical features for SMOTENC (SMOTE variant) data augmentation.

In [37]:
train_X_og.columns

Index(['transcript_position', '-1_flank_length', '-1_flank_std',
       '-1_flank_mean', 'central_length', 'central_std', 'central_mean',
       '+1_flank_length', '+1_flank_std', '+1_flank_mean'],
      dtype='object')

In [38]:
a = list(range(0, 9)) # idx of all var
non_cat = []
cat = [i for i in a if i not in non_cat]

Sample minority class using SMOTENC such that post-sampling ratio is around 1 minority:10 majority and minority class is sampled by a factor of around 5.

In [40]:
sm = SMOTENC(random_state = 4266, categorical_features = cat, sampling_strategy = 0.12)
X_train_resampled, y_train_resampled = sm.fit_resample(X_train_resampled, y_train_resampled)

KeyboardInterrupt: ignored

In [ ]:
y_train_resampled.value_counts()

Use cross validation to get subset of training data with best F1-score on validation set.

In [ ]:
# fit model
clf = LogisticRegression(random_state = 4266, max_iter = 500, class_weight = "balanced")

# CV
best_fold = get_best(X_train_resampled, y_train_resampled, clf)

Train model using subset of data identified above using cross validation and Test model using holdout set.

In [ ]:
X_train = best_fold[0]
y_train = best_fold[1]
X_test = test_X
y_test = test_y

# fit model
clf = LogisticRegression(random_state = 4266,
                         max_iter = 500, class_weight = "balanced").fit(X_train, y_train)

cfm = met.confusion_matrix(y_true = y_train, y_pred = clf.predict(X_train))
cfm2 = cfm.astype("float") / cfm.sum(axis=1)[:, np.newaxis]
train_1 = {"model": "weighted and resampled w SMOTE",
  "roc auc score": met.roc_auc_score(y_true = y_train, y_score = clf.predict(X_train)),
	"F1-score": met.f1_score(y_true = y_train, y_pred = clf.predict(X_train)),
  "precision" : met.precision_score(y_true = y_train, y_pred = clf.predict(X_train)),
  "recall": met.recall_score(y_true = y_train, y_pred = clf.predict(X_train)),
	"tn": cfm.ravel()[0],
	"fp": cfm.ravel()[1],
  "fn": cfm.ravel()[2],
  "tp": cfm.ravel()[3],
  "class 0 accuracy": cfm2.diagonal()[0],
  "class 1 accuracy ": cfm2.diagonal()[1]}
training_results = training_results.append(train_1, ignore_index = True)

cfm = met.confusion_matrix(y_true = y_test, y_pred = clf.predict(X_test))
cfm2 = cfm.astype("float") / cfm.sum(axis=1)[:, np.newaxis]
test_1 = {"model": "weighted and resampled w SMOTE",
  "roc auc score": met.roc_auc_score(y_true = y_test, y_score = clf.predict(X_test)),
	"F1-score": met.f1_score(y_true = y_test, y_pred = clf.predict(X_test)),
  "precision": met.precision_score(y_true = y_test, y_pred = clf.predict(X_test)),
  "recall": met.recall_score(y_true = y_test, y_pred = clf.predict(X_test)),
	"tn": cfm.ravel()[0],
	"fp": cfm.ravel()[1],
  "fn": cfm.ravel()[2],
  "tp": cfm.ravel()[3],
  "class 0 accuracy": cfm2.diagonal()[0],
  "class 1 accuracy ": cfm2.diagonal()[1]}
testing_results = testing_results.append(test_1, ignore_index = True)

In [ ]:
training_results

In [ ]:
testing_results